In [1]:
using LinearAlgebra
using TrajectoryOptimization
using Plots

In [2]:
#Pauli spin matrices
i2 = [0 -1; 1 0] #2x2 unit imaginary matrix
Sx = [zeros(2,2) I; I zeros(2,2)]
Sy = [zeros(2,2) -i2; i2 zeros(2,2)]
Sz = [I zeros(2,2); zeros(2,2) -I]
iSx = [zeros(2,2) i2; i2 zeros(2,2)]
iSy = [zeros(2,2) I; -I zeros(2,2)]
iSz = [i2 zeros(2,2); zeros(2,2) -i2];

In [3]:
function qubit_dynamics!(ẋ,x,u)
      ẋ .= (Sx + u.*Sz)*x
end

qubit_dynamics! (generic function with 1 method)

In [4]:
n = 4 #state dimension
m = 1 #inut dimension
model = Model(qubit_dynamics!,n,m)
model_d = rk4(model);

In [5]:
dt = 0.01 #time step
N = 101 #number of knot points

x0 = [1.0, 0, 0, 0] #initial state
xf = [1/sqrt(2), 0, 1/sqrt(2), 0] #desired final state

u0 = [randn(m) for k = 1:N-1]; #random initial guess for control inputs

#Set up quadratic objective function
Q = 10.0*Diagonal(I,n)
R = 0.001*Diagonal(I,m)
Qf = 10.0*Diagonal(I,n)
obj = LQRObjective(Q,R,Qf,xf,N);

In [6]:
#Set up input constriants
bnd = BoundConstraint(n,m,u_max=3, u_min=-3) # control limits
goal = goal_constraint(xf) # terminal constraint

constraints = Constraints(N) # define constraints at each time step
for k = 1:N-1
    constraints[k] += bnd
end
constraints[N] += goal;

In [ ]:
#Set up and solve traj. opt. problem
prob = Problem(model_d, obj, constraints=constraints, x0=x0, xf=xf, N=N, dt=dt)
initial_controls!(prob, u0) #random guess for initial controls
solver = solve!(prob, ALTROSolverOptions{Float64}());

In [ ]:
plot(prob.X,xlabel="time step",title="State Trajectory",label=["Re(x1)" "Im(x1)" "Re(x2)" "Im(x2)"])

In [ ]:
plot(prob.U,xlabel="time step",title="Control Trajectory",label="u1")